In [5]:
import cv2
import numpy as np

# Inicializar a captura de vídeo (webcam)
cap = cv2.VideoCapture(0)  # Use 0 para a webcam padrão

# Verifique se a câmera foi aberta com sucesso
if not cap.isOpened():
    print("Erro ao acessar a câmera.")
    exit()

# Ler o primeiro frame para inicializar
ret, frame1 = cap.read()
frame1_gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
frame1_gray = cv2.GaussianBlur(frame1_gray, (21, 21), 0)

print("Pressione 'q' para sair...")

while True:
    # Ler o próximo frame
    ret, frame2 = cap.read()
    if not ret:
        break

    # Converter o frame atual para escala de cinza e aplicar desfoque
    frame2_gray = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    frame2_gray = cv2.GaussianBlur(frame2_gray, (21, 21), 0)

    # Calcular a diferença entre os frames
    frame_diff = cv2.absdiff(frame1_gray, frame2_gray)

    # Aplicar um limiar para destacar as mudanças (áreas em movimento)
    _, thresh = cv2.threshold(frame_diff, 25, 255, cv2.THRESH_BINARY)

    # Dilatar a imagem para preencher lacunas nos objetos detectados
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Encontrar contornos nas áreas destacadas
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Desenhar contornos e pontos nos objetos detectados
    for contour in contours:
        if cv2.contourArea(contour) < 500:  # Ignorar pequenas mudanças
            continue

        # Obter o retângulo delimitador do contorno
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(frame2, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Calcular o ponto central do contorno
        M = cv2.moments(contour)
        if M["m00"] != 0:  # Evitar divisão por zero
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            # Desenhar o ponto central
            cv2.circle(frame2, (cx, cy), 5, (0, 0, 255), -1)

    # Exibir apenas a janela "Detecção de Movimento"
    cv2.imshow("Detecção de Movimento", frame2)

    # Atualizar o frame de referência
    frame1_gray = frame2_gray

    # Sair ao pressionar 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos
cap.release()
cv2.destroyAllWindows()


Pressione 'q' para sair...
